<img src="../../imagens/bemvindas.png">

# Musicou toda!

A partir do material [Análise de Dados e Forró](https://dandaramcsousa.github.io/2020/05/26/dados-e-forro-vol-1.html) de [Dandara Sousa](https://dandaramcsousa.github.io/about/) e [dos exemplos do Spotipy](https://github.com/plamere/spotipy/tree/master/examples), tomamos coragem e resolvemos também brincar com o SpotyPy.

> *A ideia aqui é revisar o que aprendemos explorando uma conexão com o aplicativo Spotify e a biblioteca Pandas para a análise de dados.*

## O que já vimos até aqui?

- Funções
- Condicionais
- Laços de repetição
- Listas e dicionários

## O que faremos hoje?

1. Preparando o terreno: [Credenciais e App](https://developer.spotify.com/dashboard/login)
2. Pesquisando *uma artista* no Spotify
3. Pesquisando *um álbum* no Spotify
4. E *as músicas* dos álbuns?
5. Pegandos nossas informações no Spotify

### Outros Links:
- [Documentação spotipy](spotipy.readthedocs.org/ )

### 1º Preparando o terreno 

In [ ]:
!pip install spotipy

In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import sys
import csv

client_id="Seu Client ID"
client_secret="Seu Secret ID"
                 
sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials(client_id, client_secret))

### 2º Pesquisando uma artista

In [ ]:
results = sp.search("Chave de busca")
type(results)

*Como podemos saber o que tem dentro desse dicionário?*

In [ ]:
results.keys()

In [ ]:
results.get('tracks')

In [ ]:
results.get('tracks').keys()

In [ ]:
results.get('tracks').get("items").get('keys')

In [ ]:
results.get('tracks').get("items")[0].get('artists')

*E se quisermos fazer outra busca agora? Facilita se criarmos uma função, né?*

In [ ]:
def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results.get('artists').get('items')
    
    return items[0] if len(items) else None

*Como a execução a seguir funciona?*

In [ ]:
get_artist("Sorriso Maroto").get('genres')

### 3º Pesquisando um album no Spotify

In [ ]:
id = get_artist("Sorriso Maroto").get('id')

In [ ]:
albums = sp.artist_albums(id, album_type='album')

In [ ]:
albums.get('items')[0].keys()

In [ ]:
search = 'Por Você Ao Vivo'
full_albumn = {}

for album in albums.get('items'):
    if album.get('name') == search:
        full_albumn = album
        print(full_albumn)
        break

In [ ]:
full_albumn.get('id')

*Como a gente transforma isso em uma função?*

#### #ColaboraAquiMana

In [ ]:
def ...

### 4º E as músicas dos álbuns?

In [ ]:
tracks = sp.album_tracks(album_id='0nax8M9StAXGx54wYprGHN')
type(tracks)

In [ ]:
tracks.get('items')[0].keys()

### *é agora hein...*

In [235]:
for track in tracks.get('items'):
    print(track.get('name'))

Ainda Gosto de Você - Me Espera - Coração Deserto
Engano
Já Era
Eu Vacilei
Estrela Maior
E Agora? - Cadê Você
Adivinha O Quê?
Me Olha Nos Olhos
Diz Que Quer Ficar
Disfarça
Por Você
Pra Sempre
Aprendi e Quero Amar
Nada de Pensar Em Despedida - Marque um Lugar - Preciso Viver


<img src="https://media.giphy.com/media/6Q3M4BIK0lX44/giphy.gif">

In [ ]:
import pandas as pd

por_voce_list = []

for track in tracks.get('items'):
    track_dict = {
        'name': track.get('name'),
        'duration_ms': track.get('duration_ms')
    }
    por_voce_list.append(track_dict)
    
por_voce_df = pd.DataFrame(por_voce_list)
por_voce_df.head()

*Agora que temos essa "tabela" o pandas nos permite ordenar os valores com facilidade.*

In [ ]:
por_voce_df.sort_values('duration_ms', ascending=False)

### 5º Pegandos nossas informações no Spotify

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri="https://localhost/",
                                               scope="user-top-read"))

In [ ]:
tracks_short_term = sp.current_user_top_tracks(time_range='short_term', limit=10)
tracks_medium_term = sp.current_user_top_tracks(time_range='medium_term', limit=10)
tracks_long_term = sp.current_user_top_tracks(time_range='long_term', limit=10)

In [ ]:
my_musics = tracks_short_term.get('items') + tracks_medium_term.get('items') + tracks_long_term.get('items')

In [ ]:
my_musics[0].keys()

In [ ]:
my_musics_df = pd.json_normalize(my_musics)
my_musics_df.head()

*E se fosse com o `my_musics_df = pd.DataFrame(my_musics)`?*  

#### #ColaboraAquiMana: Adiciona o exemplo de como seria e uma descrição sobre a diferença entre as duas formas `pd.json_normalize()` e `pd.DataFrame()`.

In [ ]:
...

---

### *O que o Pandas tem a oferecer?*

#### Filtrando linhas

In [ ]:
my_musics_df.loc[my_musics_df['name'] == 'Nome']

#### Agrupando dados

In [ ]:
my_musics_df.groupby(['name', 'id']).size().reset_index(name='qtd')

#### Filtrando com o query

In [ ]:
my_musics_df.groupby(['name', 'id']).size().reset_index(name='qtd').query('qtd > 1')

#### Filtrando com o Lambda 

#### #ColaboraAquiMana: Tenta fazer esse filtro usando o *lambda* - pesquisa por "apply and Lambda usage in pandas".

In [ ]:
...

#### Lidando com registros repetidos

In [ ]:
len(my_musics_df)

In [ ]:
my_musics_df = my_musics_df.drop_duplicates('id')
len(my_musics_df)

#### Ordenando e exibindo dados

In [ ]:
my_musics_df.columns

In [ ]:
my_musics_df.sort_values('popularity', ascending=False)[['name', 'popularity', 'album.name', 'duration_ms']]

In [ ]:
!pip install altair

In [ ]:
import altair as alt

popularidade = my_musics_df.sort_values('popularity', ascending = False)

alt.Chart(popularidade).mark_circle(color = '#FF9138', opacity = 1).encode(
    y = alt.Y('name', axis = alt.Axis(title='Música'), sort = None),
    x = alt.X('popularity', axis = alt.Axis(title = "Popularidade")),
    tooltip = ['name', 'popularity', 'album.name'])

## É isso! Obrigada! <3

<img src="https://media.giphy.com/media/l4Ep71LWjYR1eCPXq/giphy.gif">